In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime,timedelta
import os
import pickle
import codecs
import re
from tqdm import tqdm_notebook as tqdm
import plotly.express as px


#For tokenizing sentences
import nltk
nltk.download('punkt')


from sklearn.decomposition import PCA

# Cte
BASE_PATH = "..\..\Datathon-2022-data\data"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\js_ma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Economic Index Ca

In [69]:
def stretch_to_daybyday(df):
    df_tmp = df.copy(deep=True)

    row_iterator = df.iterrows()
    _, last = next(row_iterator)  # take first item from row_iterator
    for i, row in row_iterator:
        try:
            date_diff = (row.date - last.date).days
                # duplace row for the difference
            tmp =  pd.DataFrame([last])
            tmp =  pd.concat([tmp]* (date_diff-1), ignore_index=True)

            # update the date or each to make it increase
            for j, tmp_row in tmp.iterrows():
                tmp.loc[j, 'date']= tmp_row.date + timedelta(days=j+1)
                

            df_tmp = pd.concat([df_tmp,  tmp], ignore_index=True)

            last = row
        except:
            last = row


    df = df_tmp.sort_values(by='date', ignore_index=True )

    return df

In [70]:
CA_interest_rate_df = pd.read_csv(f"{BASE_PATH}\dataset\EconomicIndexCa\CA_Bank_Rate.csv")

CA_interest_rate_df['date'] = pd.to_datetime(CA_interest_rate_df['date'], format='%B %Y')

CA_interest_rate_df = CA_interest_rate_df.loc[CA_interest_rate_df['bank_rate'] != '..']

CA_interest_rate_df = CA_interest_rate_df.sort_values(by='date')

CA_interest_rate_df.head()

date bank_rate
12 1935-01-01      2.50
13 1935-02-01      2.50
14 1935-03-01      2.50
15 1935-04-01      2.50
16 1935-05-01      2.50

In [71]:
CA_interest_rate_df = stretch_to_daybyday(CA_interest_rate_df)

CA_interest_rate_df.head(100)

date bank_rate
0  1935-01-01      2.50
1  1935-01-02      2.50
2  1935-01-03      2.50
3  1935-01-04      2.50
4  1935-01-05      2.50
..        ...       ...
95 1935-04-06      2.50
96 1935-04-07      2.50
97 1935-04-08      2.50
98 1935-04-09      2.50
99 1935-04-10      2.50

[100 rows x 2 columns]

In [72]:
bcpiWeekly_df = pd.read_csv(f"{BASE_PATH}\dataset\EconomicIndexCa\BCPI_WEEKLY.csv",  skiprows=17)

bcpiWeekly_df['date'] = pd.to_datetime(bcpiWeekly_df['date'], format='%Y-%m-%d')

bcpiWeekly_df = bcpiWeekly_df.sort_values(by='date')


bcpiWeekly_df.head()

date  W.BCPI  W.BCNE  W.ENER  W.MTLS  W.FOPR  W.AGRI  W.FISH
0 1972-01-12  100.00  100.00  100.00  100.00  100.00  100.00  100.00
1 1972-01-19  100.00  100.00  100.00  100.00  100.00  100.00  100.00
2 1972-01-26  100.00  100.00  100.00  100.00  100.00  100.00  100.00
3 1972-02-02  100.16  100.21   99.91  100.29  100.05  100.48   95.56
4 1972-02-09  100.40  100.53   99.78  100.74  100.12  101.20   88.89

In [73]:
bcpiWeekly_df = stretch_to_daybyday(bcpiWeekly_df)


bcpiWeekly_df.head()

date  W.BCPI  W.BCNE  W.ENER  W.MTLS  W.FOPR  W.AGRI  W.FISH
0 1972-01-12   100.0   100.0   100.0   100.0   100.0   100.0   100.0
1 1972-01-13   100.0   100.0   100.0   100.0   100.0   100.0   100.0
2 1972-01-14   100.0   100.0   100.0   100.0   100.0   100.0   100.0
3 1972-01-15   100.0   100.0   100.0   100.0   100.0   100.0   100.0
4 1972-01-16   100.0   100.0   100.0   100.0   100.0   100.0   100.0

In [74]:
COCWeights_df = pd.read_csv(f"{BASE_PATH}\dataset\EconomicIndexCa\Crude_Oil_Component_Weights.csv",  skiprows=13)[:-3]

COCWeights_df['date'] = pd.to_datetime(COCWeights_df['date'], format='%Y-%m-%d')

COCWeights_df.head()

date      WGTS.AGRI    WGTS.BRENT     WGTS.COAL     WGTS.FISH  \
0 1972-01-01  30.0966438349  0.1339349726  1.1830782638  0.9205453099   
1 1973-01-01  33.7525616330  0.1373830643  1.0110418989  1.0107277333   
2 1974-01-01  30.1812160543  0.1716956415  1.2944243234  0.6878843365   
3 1975-01-01  31.3569185412  0.1719912842  2.3595358547  0.5520468188   
4 1976-01-01  27.6891121154  0.1634566927  2.2248001630  0.7647244924   

       WGTS.FOPR      WGTS.MTLS WGTS.NATURALGAS      WGTS.WCC       WGTS.WTI  
0  30.4445473220  20.6185714306    2.9261469058  4.4042025519   9.2723294084  
1  28.4805362026  19.1857513747    2.3933703708  4.5175866363   9.5110410861  
2  27.6552621229  19.5158423862    2.9612795047  5.6458919411  11.8865036894  
3  23.1555884234  18.4900389850    6.3512954368  5.6556136003  11.9069710556  
4  26.1170564584  17.4992503426    8.8505097392  5.3749694271  11.3161205692

In [75]:
COCWeights_df = stretch_to_daybyday(COCWeights_df)

COCWeights_df.head()

date      WGTS.AGRI    WGTS.BRENT     WGTS.COAL     WGTS.FISH  \
0 1972-01-01  30.0966438349  0.1339349726  1.1830782638  0.9205453099   
1 1972-01-02  30.0966438349  0.1339349726  1.1830782638  0.9205453099   
2 1972-01-03  30.0966438349  0.1339349726  1.1830782638  0.9205453099   
3 1972-01-04  30.0966438349  0.1339349726  1.1830782638  0.9205453099   
4 1972-01-05  30.0966438349  0.1339349726  1.1830782638  0.9205453099   

       WGTS.FOPR      WGTS.MTLS WGTS.NATURALGAS      WGTS.WCC      WGTS.WTI  
0  30.4445473220  20.6185714306    2.9261469058  4.4042025519  9.2723294084  
1  30.4445473220  20.6185714306    2.9261469058  4.4042025519  9.2723294084  
2  30.4445473220  20.6185714306    2.9261469058  4.4042025519  9.2723294084  
3  30.4445473220  20.6185714306    2.9261469058  4.4042025519  9.2723294084  
4  30.4445473220  20.6185714306    2.9261469058  4.4042025519  9.2723294084

## Concat dataframe

In [76]:
indices_df = pd.concat([ CA_interest_rate_df.set_index('date'), bcpiWeekly_df.set_index('date'),COCWeights_df.set_index('date')], axis=1, join='inner')
indices_df.head()

bank_rate  W.BCPI  W.BCNE  W.ENER  W.MTLS  W.FOPR  W.AGRI  W.FISH  \
date                                                                           
1972-01-12      4.75   100.0   100.0   100.0   100.0   100.0   100.0   100.0   
1972-01-13      4.75   100.0   100.0   100.0   100.0   100.0   100.0   100.0   
1972-01-14      4.75   100.0   100.0   100.0   100.0   100.0   100.0   100.0   
1972-01-15      4.75   100.0   100.0   100.0   100.0   100.0   100.0   100.0   
1972-01-16      4.75   100.0   100.0   100.0   100.0   100.0   100.0   100.0   

                WGTS.AGRI    WGTS.BRENT     WGTS.COAL     WGTS.FISH  \
date                                                                  
1972-01-12  30.0966438349  0.1339349726  1.1830782638  0.9205453099   
1972-01-13  30.0966438349  0.1339349726  1.1830782638  0.9205453099   
1972-01-14  30.0966438349  0.1339349726  1.1830782638  0.9205453099   
1972-01-15  30.0966438349  0.1339349726  1.1830782638  0.9205453099   
1972-01-16  30.0966438349  0.1339349726  1.1830782638  0.9205453099   

                WGTS.FOPR      WGTS.MTLS WGTS.NATURALGAS      WGTS.WCC  \
date                                                                     
1972-01-12  30.4445473220  20.6185714306    2.9261469058  4.4042025519   
1972-01-13  30.4445473220  20.6185714306    2.9261469058  4.4042025519   
1972-01-14  30.4445473220  20.6185714306    2.9261469058  4.4042025519   
1972-01-15  30.4445473220  20.6185714306    2.9261469058  4.4042025519   
1972-01-16  30.4445473220  20.6185714306    2.9261469058  4.4042025519   

                WGTS.WTI  
date                      
1972-01-12  9.2723294084  
1972-01-13  9.2723294084  
1972-01-14  9.2723294084  
1972-01-15  9.2723294084  
1972-01-16  9.2723294084

# PCA

In [77]:
pca = PCA(n_components='mle')
indices_df['pca_vec'] = pca.fit_transform(indices_df.to_numpy()).tolist()
indices_df.pca_vec

date
1972-01-12    [-985.6132218887255, 68.64838565710974, -46.79...
1972-01-13    [-985.6132218896594, 68.64838565742653, -46.79...
1972-01-14    [-985.6132218897642, 68.64838565733085, -46.79...
1972-01-15    [-985.613221889766, 68.64838565736844, -46.795...
1972-01-16    [-985.6132218897719, 68.64838565737358, -46.79...
                                    ...                        
2021-12-28    [1300.3758287038875, -585.512619281244, 205.10...
2021-12-29    [1370.637879366154, -540.7048303235, 209.95353...
2021-12-30    [1370.637879366154, -540.7048303235, 209.95353...
2021-12-31    [1370.637879366154, -540.7048303235, 209.95353...
2022-01-01    [1370.7172570035573, -540.6940914561211, 209.8...
Freq: D, Name: pca_vec, Length: 18253, dtype: object

In [78]:
indices_df.to_csv("./out/CA_indices_PCA.csv", index=True)